In [ ]:
import pandas as pd

In [28]:
def find_match_LR(df,template, poids):
    """
    Calcule les scores pour un DataFrame contenant des colonnes discrètes et continues.

    Args:
    df (DataFrame): DataFrame contenant les données.
    poids_continu (float): Poids pour les colonnes continues.
    poids_discret (float): Poids pour les colonnes discrètes.

    Returns:
    DataFrame: DataFrame avec les scores calculés et la colonne 'score global'.
    """
    # Cloner le DataFrame d'origine
    df_scores = df.copy()

    # Calculer les scores pour chaque colonne
    for colonne in poids:
        if df[colonne].dtype == 'object' or df[colonne].dtype =='int64':
            # Si la colonne est discrète
            df_scores[f'score_{colonne}'] = df[colonne].apply(score_proximite_discret, args=(template[colonne],)) * poids[colonne]
        else:
            # Si la colonne est continue
            # Calculer la distance maximale pour la colonne continue
            max_distance = max(abs(df[colonne] - template[colonne]))

            df_scores[f'score_{colonne}'] = df[colonne].apply(score_proximite, args=(template[colonne],max_distance)) * poids[colonne]

    # Calculer le score global
    df_scores['score_global'] = df_scores[[col for col in df_scores.columns if col.startswith('score_')]].sum(axis=1)

    # Retourner la ligne avec le plus grand score global
    ligne_max_score = df_scores.loc[df_scores['score_global'].idxmax()]

    return df_scores, ligne_max_score

def score_proximite_discret(valeur, template):
    """
    Calcule le score de proximité d'une valeur par rapport au template de manière discrète.

    Args:
    valeur (str or int): Valeur discrète.
    template (str or int): Valeur template.

    Returns:
    float: Score de proximité (1 si la valeur est exactement la même que le template, sinon 0).
    """
    score = 1 if valeur == template else 0
    return score

def score_proximite(valeur, template, max_distance):
    """
    Calcule le score de proximité d'une valeur par rapport au template.

    Args:
    valeur (float): Valeur numérique.
    template (float): Valeur template.

    Returns:
    float: Score de proximité.
    """
    distance = abs(valeur - template)
    score = 1 - distance / max_distance
    return score

def get_poids(path = "settings.xlsx"): 

    df = pd.read_excel(path)

    dictionnaire_colonnes = df.iloc[0].to_dict()
    if 'id' in dictionnaire_colonnes:
        dictionnaire_colonnes.pop('id')
    if 'type_appart' in dictionnaire_colonnes:
        dictionnaire_colonnes.pop('type_appart')

    somme_valeurs = sum(dictionnaire_colonnes.values())

    return {cle: valeur / somme_valeurs for cle, valeur in dictionnaire_colonnes.items()}

def get_rank(path = "settings.xlsx"): 

    df = pd.read_excel(path)

    dictionnaire_colonnes = df.iloc[0].to_dict()
    if 'id' in dictionnaire_colonnes:
        dictionnaire_colonnes.pop('id')
    if 'type_appart' in dictionnaire_colonnes:
        dictionnaire_colonnes.pop('type_appart')

    return dictionnaire_colonnes

def find_match_F(df, template, rank):

    df_filtre = df[df["nb_murs"] == template["nb_murs"]]
    ranked = 2
    while len(df_filtre) > 1 and ranked < 11: 
        df_filtre2 = df_filtre.copy()
        for cle, val in rank.items():
            if val == ranked:
                df_filtre2 = df_filtre2[df_filtre2[cle] == template[cle]]
        ranked +=1
        if len(df_filtre2) >=1 : df_filtre = df_filtre2.copy()
    return df_filtre

def find_match_IF(df, template): 

    df_filtre = df[df["nb_murs"] == template["nb_murs"]]
    df_filtre = df_filtre[df_filtre["pos_f1"] == template["pos_f1"]]
    df_filtre = df_filtre[(df_filtre['pos_f2'] >= template["pos_f2"]*0.9) & (df_filtre['pos_f2'] <= template["pos_f2"]*1.1)]


    return df_filtre


In [ ]:
df = pd.read_excel('appartements.xlsx')
rank = get_rank()
template = df.iloc[0].to_dict()
find_match_IF(df, template)

In [ ]:
df = pd.read_excel('appartements.xlsx')
rank = get_rank()
template = df.iloc[0].to_dict()
find_match_F(df, template, rank)

In [ ]:
df = pd.read_excel('appartements.xlsx')
poids = get_poids()
template = df.iloc[0].to_dict()
find_match_LR(df, template, poids)

In [1]:
from scipy.spatial.distance import directed_hausdorff

def similarity_score(shape1, shape2):
    """
    Calcule le score de similarité entre deux formes géométriques.

    Args:
    shape1 (list): Liste de points représentant la première forme.
    shape2 (list): Liste de points représentant la deuxième forme.

    Returns:
    float: Score de similarité entre les deux formes.
    """
    # Calculer la distance de Hausdorff entre les deux formes
    distance1 = directed_hausdorff(shape1, shape2)[0]
    distance2 = directed_hausdorff(shape2, shape1)[0]

    # Score de similarité basé sur la moyenne des deux distances de Hausdorff
    similarity_score = (distance1 + distance2) / 2

    return similarity_score

# Exemple d'utilisation
shape1 = [(0, 0), (1, 1), (2, 0)]
shape2 = [(0, 1), (1, 0), (2, 1)]

score = similarity_score(shape1, shape2)
print("Score de similarité entre les deux formes :", score)


Score de similarité entre les deux formes : 1.0
